In [1]:
import pandas as pd
import numpy as np
import requests
import time
import pprint

#API key pulled from a config.py in format of `prog_search_key` =  "your_key_here" 
#from config import prog_search_key

# Dataset Transformation

### <center> Attempt using country_codes_combined.csv </center>

In [2]:
country_codes_csv = pd.read_csv('static/data/country_codes_combined.csv')
country_codes_csv = country_codes_csv[ ['alpha2','de','en'] ]
country_codes_df = country_codes_csv.copy()
country_codes_df.head()

,alpha2,de,en
0,af,Afghanistan,Afghanistan
1,al,Albanien,Albania
2,dz,Algerien,Algeria
3,ad,Andorra,Andorra
4,ao,Angola,Angola


In [3]:
world_risk_index_csv = pd.read_csv('static/data/world_risk_index.csv')

In [4]:
## Found during transformation was this mis-input row. It must be manually adjusted or deleted.
world_risk_index_csv.loc[[1858]]

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category
1858,Korea Republic of 4.59,14.89,30.82,14.31,46.55,31.59,NaN,2016,Very High,Very High,NaN,High


In [5]:
# Because the entire index of 1858 was misinput I manually adjust the row across each column. Parameters for 
## Category found from supporting document from data creators.

korea_row_fix = ['Korea, Republic of', 4.59, 14.89, 30.82, 14.31, 46.55, 31.59,2016,'High','Low','Very Low','Very Low']

world_risk_index_csv.loc[[1858]] = korea_row_fix
world_risk_index_csv.loc[[1858]] = world_risk_index_csv.loc[[1858]].copy()
world_risk_index_csv.loc[[1858]]

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category
1858,"Korea, Republic of",4.59,14.89,30.82,14.31,46.55,31.59,2016,High,Low,Very Low,Very Low


In [6]:
# Get length of index as a reference to ensure consistent dataframe size through-out. 

print(f'Rows in country_codes_df: {len(country_codes_df.index)}')
print(f'Rows in country_codes_df: {len(world_risk_index_csv.index)}')

Rows in country_codes_df: 193
Rows in country_codes_df: 1917


In [7]:
# Making two dataframes based on imported English and German country codes & region names. These two dataframes are then
## merged together into one dataframe so there is an English reference point to the German region names.

merged_df_en = world_risk_index_csv.merge(country_codes_df, how='left', left_on='Region', right_on='en')
merged_df_de = world_risk_index_csv.merge(country_codes_df, how='left', left_on='Region', right_on='de')
merged_all = pd.concat([merged_df_en, merged_df_de])
merged_dropped = merged_all.dropna().copy()
merged_world = merged_dropped.merge(world_risk_index_csv,how='right')
merged_final = merged_world.drop_duplicates(ignore_index=True).copy()
merged_final

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category,alpha2,de,en
0,Vanuatu,32.00,56.33,56.81,37.14,79.34,53.96,2011,Very High,Very High,High,High,vu,Vanuatu,Vanuatu
1,Tonga,29.08,56.04,51.90,28.94,81.80,44.97,2011,Very High,Very High,Medium,Medium,to,Tonga,Tonga
2,Philippinen,24.32,45.09,53.93,34.99,82.78,44.01,2011,Very High,Very High,High,High,ph,Philippinen,Philippines
3,Salomonen,23.51,36.40,64.60,44.11,85.95,63.74,2011,Very High,Very High,Very High,High,sb,Salomonen,Solomon Islands
4,Guatemala,20.88,38.42,54.35,35.36,77.83,49.87,2011,Very High,Very High,High,High,gt,Guatemala,Guatemala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,Grenada,1.42,3.13,45.39,24.54,68.82,42.82,2016,Very Low,Very Low,Medium,Medium,gd,Grenada,Grenada
1913,Barbados,1.32,3.46,38.26,18.20,50.29,46.29,2016,Very Low,Very Low,Low,Low,bb,Barbados,Barbados
1914,Saudi Arabia,1.14,2.93,38.96,14.80,65.01,37.07,2016,Very Low,Very Low,Low,Very Low,sa,Saudi-Arabien,Saudi Arabia
1915,Malta,0.60,1.65,36.25,15.97,59.33,33.44,2016,Very Low,Very Low,Low,Very Low,mt,Malta,Malta


In [8]:
# A check to see if there are any Region inputs that do not have a corresponding alpha2 code, which is the 
## consistent method used for our translation. The German region name and the English region name will always share the
### same unified country code. 

merged_final_nulls = merged_final[merged_final['alpha2'].isnull()]
merged_final_nulls_drop = merged_final_nulls.drop_duplicates(subset=['Region']).copy()

print(f' Length of null values without duplicates removed: {len(merged_final_nulls.index)}')
print(f' Length of null values with duplicates removed: {len(merged_final_nulls_drop.index)}')

 Length of null values without duplicates removed: 129
 Length of null values with duplicates removed: 44


###  <center> Attempt at Pulling From Additional Source </center>

As there are 130 rows that show up with lack of information from the first source two additional websites were pulled from, one in English and one in German. The desire was that this would provide coverage that may have been missed from the original `country_codes_combined.csv` 
    
English Source: [Cloford.com](https://cloford.com/resources/codes/index.htm)
    
German Source: [oenb.at](https://www.oenb.at/Statistik/Klassifikationen/ISO-Codes/ISO-Code-Verzeichnis-fuer-Laender--und-Waehrungscodes.html)    

    


In [9]:
# Code to pull tables from English [country:country_code] code source

url = "https://cloford.com/resources/codes/index.htm"

country_code_import = pd.read_html(url)
country_code_draft = country_code_import[3].copy()
country_code_df = country_code_draft[  ['Country','ISO (2)','Continent','Region','Capital' ]  ]
country_code_df.head()

,Country,ISO (2),Continent,Region,Capital
0,Afghanistan,AF,Asia,South Asia,Kabul
1,Albania,AL,Europe,South East Europe,Tirana
2,Algeria,DZ,Africa,Northern Africa,Algiers
3,American Samoa,AS,Oceania,Pacific,Pago Pago
4,Andorra,AD,Europe,South West Europe,Andorra la Vella


In [13]:
# Code to pull tables from German [country:country_code] code source

ger_url = "https://www.oenb.at/Statistik/Klassifikationen/ISO-Codes/ISO-Code-Verzeichnis-fuer-Laender--und-Waehrungscodes.html"
ger_codes = pd.read_html(ger_url)
ger_code_draft = ger_codes[0].copy()
ger_code_df = ger_code_draft[ ['Land','ISO-Code (Land)'] ]
ger_code_df = ger_code_df.fillna("").copy()
ger_code_df_clean =  ger_code_df.loc[ger_code_df['ISO-Code (Land)']!='一一一']
ger_code_df_clean.head()

,Land,ISO-Code (Land)
0,Afghanistan,AF
1,Ägypten,EG
2,Aland,AX
3,Albanien,AL
4,Algerien,DZ


In [14]:
# Merge both German and English external sources to apply to primary dataset.

merged_import_codes = ger_code_df_clean.merge(country_code_df, left_on='ISO-Code (Land)', right_on='ISO (2)').copy()
merged_import_codes_rename= merged_import_codes.rename(columns={'Region':'Area'}).copy()
merged_import_codes_rename.head()

,Land,ISO-Code (Land),Country,ISO (2),Continent,Area,Capital
0,Afghanistan,AF,Afghanistan,AF,Asia,South Asia,Kabul
1,Ägypten,EG,Egypt,EG,Africa,Northern Africa,Cairo
2,Albanien,AL,Albania,AL,Europe,South East Europe,Tirana
3,Algerien,DZ,Algeria,DZ,Africa,Northern Africa,Algiers
4,Andorra,AD,Andorra,AD,Europe,South West Europe,Andorra la Vella


In [15]:
# Merge combined external resource dataframes with the primary dataset

merged_df_import_de = world_risk_index_csv.merge(merged_import_codes_rename, how='left', left_on='Region', right_on='Land')
merged_df_import_en = world_risk_index_csv.merge(merged_import_codes_rename, how='left', left_on='Region', right_on='Country')
import_merged_all = pd.concat([merged_df_import_en, merged_df_import_de])
import_merged_dropped = import_merged_all.dropna().copy()
import_merged_world = import_merged_dropped.merge(world_risk_index_csv,how='right')
import_merged_final = import_merged_world.drop_duplicates(ignore_index=True).copy()
import_merged_final

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category,Land,ISO-Code (Land),Country,ISO (2),Continent,Area,Capital
0,Vanuatu,32.00,56.33,56.81,37.14,79.34,53.96,2011,Very High,Very High,High,High,Vanuatu,VU,Vanuatu,VU,Oceania,Pacific,Port-Vila
1,Tonga,29.08,56.04,51.90,28.94,81.80,44.97,2011,Very High,Very High,Medium,Medium,Tonga,TO,Tonga,TO,Oceania,Pacific,Nuku'alofa
2,Philippinen,24.32,45.09,53.93,34.99,82.78,44.01,2011,Very High,Very High,High,High,Philippinen,PH,Philippines,PH,Asia,South East Asia,Manila
3,Salomonen,23.51,36.40,64.60,44.11,85.95,63.74,2011,Very High,Very High,Very High,High,Salomonen,SB,Solomon Islands,SB,Oceania,Pacific,Honiara
4,Guatemala,20.88,38.42,54.35,35.36,77.83,49.87,2011,Very High,Very High,High,High,Guatemala,GT,Guatemala,GT,Americas,Central America,Guatemala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,Grenada,1.42,3.13,45.39,24.54,68.82,42.82,2016,Very Low,Very Low,Medium,Medium,Grenada,GD,Grenada,GD,Americas,West Indies,Saint George's
1913,Barbados,1.32,3.46,38.26,18.20,50.29,46.29,2016,Very Low,Very Low,Low,Low,Barbados,BB,Barbados,BB,Americas,West Indies,Bridgetown
1914,Saudi Arabia,1.14,2.93,38.96,14.80,65.01,37.07,2016,Very Low,Very Low,Low,Very Low,Saudi-Arabien,SA,Saudi Arabia,SA,Asia,South West Asia,Riyadh
1915,Malta,0.60,1.65,36.25,15.97,59.33,33.44,2016,Very Low,Very Low,Low,Very Low,Malta,MT,Malta,MT,Europe,Southern Europe,Valletta


In [16]:
# Determine how much coverage was provided by external resource

import_final_nulls = import_merged_final[import_merged_final['ISO (2)'].isnull()]
import_final_nulls_drop = import_final_nulls.drop_duplicates(subset=['Region']).copy()

print(f' Length of null values without duplicates removed: {len(import_final_nulls.index)}')
print(f' Length of null values with duplicates removed: {len(import_final_nulls_drop.index)}')

 Length of null values without duplicates removed: 217
 Length of null values with duplicates removed: 55


### <center> Combine csv source dataframe with externally pulled source dataframe </center>

In [17]:
import_csv_merge = import_merged_final.merge(merged_final,how='outer').copy()

In [18]:
# Create dataframe out of both ISO code fields from each source.
iso_codes_df = import_csv_merge[ ['ISO (2)','alpha2'] ]

## Where ISO 2 is null I want to find an alpha2 row that is not null so that I can make sure that the backfill I use in the
### next cell works properly. 
iso_codes_df[iso_codes_df['ISO (2)'].isnull()].head(1)

,ISO (2),alpha2
6,NaN,tl


In [19]:
import_csv_merge['iso_code'] = iso_codes_df.bfill(axis=1).iloc[:, 0]
import_csv_merge.loc[import_csv_merge['iso_code'] == "tl"].head(2)

# The bfill works properly. Where there is a NULL in ISO (2) iso_code and alpha2 are filled. Performing the inverse
## will confirm that everything backfilled properly and we have the correct amount of leftover NULL codes that were not
### filled at all. 

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,...,ISO-Code (Land),Country,ISO (2),Continent,Area,Capital,alpha2,de,en,iso_code
6,Timor-Leste,17.45,25.97,67.17,52.42,89.16,59.93,2011,Very High,Very High,...,NaN,NaN,NaN,NaN,NaN,NaN,tl,Osttimor,Timor-Leste,tl
182,Timor-Leste,16.37,25.73,63.61,52.16,79.36,59.31,2013,Very High,Very High,...,NaN,NaN,NaN,NaN,NaN,NaN,tl,Osttimor,Timor-Leste,tl


In [20]:
iso_codes_df[iso_codes_df['alpha2'].isnull()].head(1)

,ISO (2),alpha2
67,CG,NaN


In [21]:
import_csv_merge['iso_code'] = iso_codes_df.bfill(axis=1).iloc[:, 0]
import_csv_merge.loc[import_csv_merge['iso_code'] == "CG"].head(2)

# As expected, where alpha2 is null, both iso_code and ISO (2) are not null. 

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,...,ISO-Code (Land),Country,ISO (2),Continent,Area,Capital,alpha2,de,en,iso_code
67,Kongo,7.71,12.19,63.28,50.98,87.39,51.45,2011,Medium,Medium,...,CG,"Congo, Republic of the",CG,Africa,Central Africa,Brazzaville,NaN,NaN,NaN,CG
236,Kongo,7.57,11.65,64.96,55.63,85.28,53.99,2013,Medium,High,...,CG,"Congo, Republic of the",CG,Africa,Central Africa,Brazzaville,NaN,NaN,NaN,CG


In [22]:
merged_source_nulls = import_csv_merge[import_csv_merge['iso_code'].isnull()]
merged_source_nulls_drop = merged_source_nulls.drop_duplicates(subset=['Region']).copy()

print(f' Length of null values without duplicates removed: {len(merged_source_nulls.index)}')
print(f' Length of null values with duplicates removed: {len(merged_source_nulls_drop.index)}')

 Length of null values without duplicates removed: 92
 Length of null values with duplicates removed: 32


In [23]:
# null_codes = import_csv_merge[import_csv_merge['iso_code'].isnull()].copy()
# null_codes['Region'] = null_codes['Region'].drop_duplicates().copy()
# null_codes = null_codes[null_codes['Region'].notna()]
# # null_codes['Region'] = null_codes['Region'].str.replace('\d+', '')
# null_codes = null_codes.reset_index()
# null_codes

## Attempt to finish cleaning by using custom google search API

Finally, after trying to use two different sources for screening German names I found out that the original data input from the original dataset source was inconsistent. Following is how I solved this problem using Google's Custom Search API. 

Fortunately I was able to cut the amount of inconsistent naming conventions down to 33 unique countries. This fits within the Custom Search API's 100 free daily search limits. 

In [24]:
null_codes_list = []
null_codes_list_comp = []
null_codes_list = merged_source_nulls_drop['Region'].tolist()
null_codes_list = [
"Vereinigte Staaten von Amerika" if ('Vereinigte Staaten v. A.') in country else country for country in null_codes_list
].copy()
null_codes_list

['Swasiland',
 'Mazedonien',
 'Tschechische Republik',
 'Südkorea',
 'Vereinigte Arabisch Emirate',
 'Vereinigte Staaten von Amerika',
 'Surinam',
 'Moldawien',
 'Vereinigte Staaten von Amerika',
 'Weißrussland',
 'Zentralafrik. Republik',
 'Ver. Arabische Emirate',
 'United Republic of Tanzania',
 'T. f. Yugo. Rep. of Macedonia',
 'Republic of Moldova',
 'Korea Republic of',
 'Libyan Arab Jamahiriya',
 'Demokratische Rep. Kongo',
 'Föd. Staaten von Mikronesien',
 'Ver. Staaten von Amerika',
 'São Tomé and Príncipe',
 'St. Vincent u. die Grenadinen',
 'Österreich',
 'Deutschland',
 'Norwegen',
 'St. Vincent und d. Grenadinen',
 'Föd. Staaten v. Mikronesien',
 'St. Vincent u. d. Grenadinen',
 'Romänien',
 'Mongolien',
 'Republic of Macedonia',
 "Lao People's Democ. Republic"]

In [25]:
### Retroactively, after performing the first API call, these countries needed to be manually transformed in order to 
### utilize the API. A follow up API called was performed on the transformed named and the results came back correctly.
### Since it has been confirmed to work, I have retroactively changed them before the first API call so that for any 
### additional use of this notebook only requires 1 API call. I have not removed the code for the 2nd API call. I have 
### simply commented it out. 


null_codes_list_comp = [
    "North Macedonia" if country == 'T. f. Yugo. Rep. of Macedonia' else country for country in null_codes_list
].copy()

null_codes_list_comp = [
    "United States" if ('Vereinigte Staaten') in country else country for country in null_codes_list_comp
].copy()


null_codes_list_comp = [
    "Central African Republic" if ('Zentralafrik') in country else country for country in null_codes_list_comp
].copy() 

null_codes_list_comp = [
    "United Arab Emirates" if ('Arabische Emirate') in country else country for country in null_codes_list_comp
].copy() 

null_codes_list_comp = [
    "Democratic Republic of the Congo" if ('Kongo') in country else country for country in null_codes_list_comp
].copy() 

null_codes_list_comp = [
    "Federated States of Micronesia" if ('Mikronesien') in country else country for country in null_codes_list_comp
].copy()

null_codes_list_comp = [
    "Saint Vincent and the Grenadines" if ('St. Vincent') in country else country for country in null_codes_list_comp
].copy()
null_codes_list_comp

['Swasiland',
 'Mazedonien',
 'Tschechische Republik',
 'Südkorea',
 'Vereinigte Arabisch Emirate',
 'United States',
 'Surinam',
 'Moldawien',
 'United States',
 'Weißrussland',
 'Central African Republic',
 'United Arab Emirates',
 'United Republic of Tanzania',
 'North Macedonia',
 'Republic of Moldova',
 'Korea Republic of',
 'Libyan Arab Jamahiriya',
 'Democratic Republic of the Congo',
 'Federated States of Micronesia',
 'Ver. Staaten von Amerika',
 'São Tomé and Príncipe',
 'Saint Vincent and the Grenadines',
 'Österreich',
 'Deutschland',
 'Norwegen',
 'Saint Vincent and the Grenadines',
 'Federated States of Micronesia',
 'Saint Vincent and the Grenadines',
 'Romänien',
 'Mongolien',
 'Republic of Macedonia',
 "Lao People's Democ. Republic"]

In [26]:
# All of the manually changed regions as a dictionary. Necessary to match country codes after since the original inputs
## are being changed from how they appear. 

changed_regions_dict = {
    'English': [
    "North Macedonia","United States","United States","United States","Central African Republic","United Arab Emirates",
    "Democratic Republic of the Congo","Federated States of Micronesia","Federated States of Micronesia",
    "Saint Vincent and the Grenadines","Saint Vincent and the Grenadines","Saint Vincent and the Grenadines"],
    'German':[
        'T. f. Yugo. Rep. of Macedonia','Vereinigte Staaten v. A.','Vereinigte Staaten von Amerika',
        'Ver. Staaten von Amerika','Zentralafrik. Republik','Ver. Arabische Emirate','Demokratische Rep. Kongo',
        'Föd. Staaten von Mikronesien','Föd. Staaten v. Mikronesien','St. Vincent und d. Grenadinen',
        'St. Vincent u. d. Grenadinen','St. Vincent u. die Grenadinen']
}

changed_regions_df = pd.DataFrame.from_dict(changed_regions_dict)
changed_regions_df

,English,German
0,North Macedonia,T. f. Yugo. Rep. of Macedonia
1,United States,Vereinigte Staaten v. A.
2,United States,Vereinigte Staaten von Amerika
3,United States,Ver. Staaten von Amerika
4,Central African Republic,Zentralafrik. Republik
5,United Arab Emirates,Ver. Arabische Emirate
6,Democratic Republic of the Congo,Demokratische Rep. Kongo
7,Federated States of Micronesia,Föd. Staaten von Mikronesien
8,Federated States of Micronesia,Föd. Staaten v. Mikronesien
9,Saint Vincent and the Grenadines,St. Vincent und d. Grenadinen


In [27]:
api_call_list = []
[api_call_list.append(country) for country in null_codes_list_comp if country not in api_call_list].copy()
api_call_list

['Swasiland',
 'Mazedonien',
 'Tschechische Republik',
 'Südkorea',
 'Vereinigte Arabisch Emirate',
 'United States',
 'Surinam',
 'Moldawien',
 'Weißrussland',
 'Central African Republic',
 'United Arab Emirates',
 'United Republic of Tanzania',
 'North Macedonia',
 'Republic of Moldova',
 'Korea Republic of',
 'Libyan Arab Jamahiriya',
 'Democratic Republic of the Congo',
 'Federated States of Micronesia',
 'Ver. Staaten von Amerika',
 'São Tomé and Príncipe',
 'Saint Vincent and the Grenadines',
 'Österreich',
 'Deutschland',
 'Norwegen',
 'Romänien',
 'Mongolien',
 'Republic of Macedonia',
 "Lao People's Democ. Republic"]

In [28]:
# api_url = "https://customsearch.googleapis.com/customsearch/v1?"
# cx = "d3772df2249924485"
# key = prog_search_key
# num = 1
# site_search = "https://en.wikipedia.org/wiki/ISO_3166-2:"
# search_filter = "i"
# query_url = (f"{api_url}cx={cx}&key={key}&num={num}&{site_search}&{search_filter}&q=")

# API Call 
## Please do not try to run this cell. I have set it to read-only. 

I have also commented it out as it should ONLY be used by Jacob McManaman, or by someone who knows what they are doing (or who is aware that *thinking* they know what they are doing can easily have consequences) and has willingly set up their Google API key for use with Google's Custom Search API. Someone who has done so must also have acknowledged that there is by default only 100 searches per day. Thoughtfully, this call will only run 33 searches.

API aside, running this cell will reset the `request_list` list which *can* be something incredibly annoying. I believe I have taken steps to circumvent any accidents, but in the event I have not taken enough precaution, should someone go through the effort to change the cell from read-only and runs the cell frivolously, you will make the writer of this markdown doomingly sad.

In [29]:
# counter = 0
# request_list= []
# for country in api_call_list:
#     counter = counter + 1
#     query = requests.get(f"{query_url}{country} iso code").json()
#     print(f"Search Request {counter} of {len(api_call_list)} : {country}")
#     request_list.append(query)
#     time.sleep(.5)

 ### Reasons for and Mechanics of the API Call:
Originally, I had hoped that there was consistency with the original Dataset. I was very wrong and the German country/region names deviated from convention. Thankfully I was able to clean 98% (1884/1917) of the German region names using two external sources. 

I found that I could just google the final 2% (33) country names and google would correct the search to produce a country code provided from `de.wikipedia.org/`. A useful tool google provides is the ability to filter by website, through a **site:`www.example.com`** query, or by creating a [Programmable Search Engine](https://programmablesearchengine.google.com/about/). This programable enginge can then be utilized by [Google's Custom Search API](https://developers.google.com/custom-search/v1/overview). Limited by 100 free searches a day, this project is very fortunate that only 33 of the data needed this treatment. The overview of the API is as follows;

`https://www.googleapis.com/customsearch/v1/siterestrict?cx=   &key=   &q=`

Where `?cx=` is the engine ID that is referenced for the search, the `&key=` is the API key that is used to make the call, and `&q=` is the query. 

And so this API call utilizes a programmable engine set to specifically filter websites by `de.wikipedia.org/`. While other websites did populate, since a call needed to be made individually for each erroneous data , the german wikipedia was preferable since its results returned the single country/regions information page, while others returned a table with every other country code. Organically, the search would look something like this:

![title](data/images/organic_search.png)

Thanks to Google, any sort of cleaning of poorly inputed data is done for us by these request. It's just up to us to clean the resulting request results.

## JSON cleaning

Once the API call is done, the resulting JSON is sent to a list and that list is cleaned in this code.

In [30]:
#request_list[0]['items'][0]['link']

In [31]:
# request_url = request_list[0]['items'][0]['link']
# split_list = request_url.split(':')
# split_list[2]

In [32]:
# bad_requests = []
# good_requests = []
# all_country_codes = []
# country_codes = []
# for request in range(len(request_list)):
#     try:
#         request_url = request_list[request]['items'][0]['link']
#         split_list = request_url.split(':')
#         print(f"Country Code: {split_list[2]}")
#         all_country_codes.append(split_list[2])
#         country_codes.append(split_list[2])
#         good_requests.append(request_list[request]['queries']['request'][0]['searchTerms'])
#     except(KeyError):
#         print(f"Skipped request {request}: {request_list[request]['queries']['request'][0]['searchTerms']}")
#         bad_requests.append(request_list[request]['queries']['request'][0]['searchTerms'])
#         all_country_codes.append(request_list[request]['queries']['request'][0]['searchTerms'])

In [33]:
#change_df

In [34]:
# codes_country_df = pd.DataFrame({
#     'iso_code': country_codes,
#     'Regions': api_call_list
# })
# codes_country_df = codes_country_df.merge(changed_regions_df,how='left',left_on = 'Regions',right_on='English')
# codes_country_df['German'] = codes_country_df['German'].fillna(codes_country_df['Regions'])
# full_country_codes_df = codes_country_df[ ['German','iso_code'] ]
# full_country_codes_df.to_csv('static/data/corrected_country_codes.csv',index=False)
# full_country_codes_df

In [35]:
## Both of these lists should provide the same length of the API call and resulting cleaning of the JSON was successful.

# print(f' Length of api_call_list: {len(api_call_list)}')
# print(f' Length of full_country_codes_df: {len(full_country_codes_df.index)}')

# Second API Call 

No longer needed as explained above - was used to further refine the bad requests recieved from the first API call. All cells have been set to READ-ONLY to make sure they are not accidentally deleted.

In [36]:
# counter = 0
# fixed_request_list= []
# for country in fixed_requests:
#     counter = counter + 1
#     query = requests.get(f"{query_url}{country} iso code").json()
#     print(f"Search Request {counter} of {len(fixed_requests)} : {country}")
#     fixed_request_list.append(query)
#     time.sleep(.5)

In [37]:
# test_bad_requests = []
# test_all_country_codes = []
# test_country_codes = []
# for request in range(len(fixed_request_list)):
#     try:
#         fixed_test_url = fixed_request_list[request]['items'][0]['link']
#         split_list = fixed_test_url.split(':')
#         print(f"Country Code: {split_list[2]}")
#         test_all_country_codes.append(split_list[2])
#         test_country_codes.append(split_list[2])
#     except(KeyError):
#         print(f"Skipped request {request}: {fixed_request_list[request]['queries']['request'][0]['searchTerms']}")
#         test_bad_requests.append(fixed_request_list[request]['queries']['request'][0]['searchTerms'])
#         test_country_codes.append(fixed_request_list[request]['queries']['request'][0]['searchTerms'])

In [38]:
# iso_bad_request = []
# iso_good_request = []

# iso_bad_request = ([s.replace(' iso code', '') for s in bad_requests])

# iso_good_request = ([s.replace(' iso code', '') for s in good_requests])

In [39]:
# fixed_bad_codes_df = pd.DataFrame({
#     'alpha2': test_country_codes,
#     'Regions': iso_bad_request
# })

# fixed_good_codes_df = pd.DataFrame({
#     'alpha2': country_codes,
#     'Regions': iso_good_request
# })

# joined_codes = fixed_bad_codes_df.merge(fixed_good_codes_df, how='right').copy()


## Finish Cleaning

In [40]:
full_country_codes_csv = pd.read_csv('static/data/corrected_country_codes.csv')
full_country_codes_csv

,German,iso_code
0,Swasiland,SZ
1,Mazedonien,MK
2,Tschechische Republik,CZ
3,Südkorea,KR
4,Vereinigte Arabisch Emirate,AE
5,Vereinigte Staaten v. A.,US
6,Vereinigte Staaten von Amerika,US
7,Ver. Staaten von Amerika,US
8,Surinam,SR
9,Moldawien,MD


In [41]:
cleaned_country_df = import_csv_merge.merge(full_country_codes_csv,how='left',left_on='Region',right_on='German').copy()
cleaned_country_df['iso_code_y'] = cleaned_country_df['iso_code_y'].fillna(cleaned_country_df['iso_code_x'])
cleaned_country_df = cleaned_country_df.drop(columns='iso_code_x')
cleaned_country_df['iso_code_y'] = cleaned_country_df['iso_code_y'].fillna(cleaned_country_df['alpha2'])
cleaned_country_df = cleaned_country_df.drop(columns='alpha2')
cleaned_country_df['iso_code_y'] = cleaned_country_df['iso_code_y'].fillna(cleaned_country_df['ISO (2)'])
cleaned_country_df = cleaned_country_df.drop(columns='ISO (2)')
cleaned_country_df['iso_code_y'] = cleaned_country_df['iso_code_y'].fillna(cleaned_country_df['ISO-Code (Land)'])
cleaned_country_df = cleaned_country_df.drop(columns=
                                             ['ISO-Code (Land)','Land',
                                             'Country','Continent','Area',
                                             'Capital','de','en','German']
                                            )
cleaned_country_df[cleaned_country_df['iso_code_y'].isnull()]

,Region,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category,iso_code_y


In [42]:
cleaned_country_df = cleaned_country_df.rename(columns={'iso_code_y':'iso_code'})
cleaned_country_df['iso_code'] = cleaned_country_df['iso_code'].str.upper()
country_codes_df['alpha2'] = country_codes_df['alpha2'].str.upper()

In [77]:
english_dataframe = cleaned_country_df.merge(country_codes_df,how='left',left_on='iso_code',right_on='alpha2')
english_dataframe  = english_dataframe .drop(columns=
                                             ['Region','alpha2',
                                             'de'])

english_dataframe = english_dataframe.sort_values(by = ["Year"])
english_dataframe

,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category,iso_code,en
0,32.00,56.33,56.81,37.14,79.34,53.96,2011,Very High,Very High,High,High,VU,Vanuatu
111,5.16,9.34,55.23,43.45,80.64,41.61,2011,Very Low,Low,High,High,BO,Bolivia (Plurinational State of)
112,5.13,11.50,44.61,24.18,67.33,42.33,2011,Low,Low,Low,Medium,JO,Jordan
113,5.11,10.40,49.07,21.48,80.01,45.73,2011,Low,Low,Medium,Low,IR,Iran (Islamic Republic of)
114,5.01,11.12,45.03,21.05,70.28,43.75,2011,Low,Low,Low,Low,LB,Lebanon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,4.89,11.94,40.97,18.23,64.82,39.86,2021,Low,Low,Low,Low,SC,Seychelles
804,4.96,17.59,28.20,16.06,47.45,21.08,2021,High,Low,Very Low,Very Low,NZ,New Zealand
803,4.97,11.35,43.80,22.68,76.22,32.51,2021,Low,Low,Medium,Medium,BR,Brazil
801,5.07,15.24,33.25,16.07,58.89,24.78,2021,High,Low,Very Low,Very Low,HU,Hungary


In [44]:
## Import ISO 3 codes alongside ISO 2 codes in order to merge with dataset.

In [78]:
iso_three_url = "https://www.iban.com/country-codes"

iso_three_import = pd.read_html(iso_three_url,na_values=(""),keep_default_na=False)
iso_three_import = iso_three_import[0].copy()
iso_three_import = iso_three_import[  ['Alpha-2 code','Alpha-3 code']  ]
iso_three_import = iso_three_import.rename(columns={'Alpha-2 code':'iso_code','Alpha-3 code':'iso3_code'})

In [72]:
## Merge and replace iso 2 with iso 3

In [79]:
english_dataframe = english_dataframe.merge(iso_three_import)
english_dataframe = english_dataframe.drop(columns={'iso_code'})
english_dataframe = english_dataframe.rename(columns={'iso3_code':'iso_code'}).copy()
english_dataframe

,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Year,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category,en,iso_code
0,32.00,56.33,56.81,37.14,79.34,53.96,2011,Very High,Very High,High,High,Vanuatu,VUT
1,36.31,63.66,57.04,34.17,81.19,55.78,2012,Very High,Very High,High,High,Vanuatu,VUT
2,36.43,63.66,57.23,34.66,81.27,55.77,2013,Very High,Very High,High,High,Vanuatu,VUT
3,36.50,63.66,57.34,36.40,81.16,54.45,2014,Very High,Very High,High,High,Vanuatu,VUT
4,36.72,63.66,57.68,35.69,81.16,56.20,2015,Very High,Very High,High,High,Vanuatu,VUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,4.52,10.24,44.15,21.72,75.19,35.55,2019,Low,Low,Medium,Medium,Saint Lucia,LCA
1913,4.70,10.24,45.88,24.22,75.67,37.74,2020,Low,Low,Medium,Medium,Saint Lucia,LCA
1914,4.52,9.83,45.96,23.68,74.26,39.95,2021,Low,Low,Medium,Medium,Saint Lucia,LCA
1915,28.47,62.74,45.38,26.12,71.21,38.82,2020,Very High,Very High,Medium,Medium,Dominica,DMA


In [68]:
# Create an 'id' column for a primary key for postGRES
english_dataframe["id"] = list(range(1, len(english_dataframe) + 1))

In [69]:
english_dataframe.columns

Index(['WRI', 'Exposure', 'Vulnerability', 'Susceptibility',
       'Lack of Coping Capabilities', ' Lack of Adaptive Capacities', 'Year',
       'Exposure Category', 'WRI Category', 'Vulnerability Category',
       'Susceptibility Category', 'en', 'iso_code', 'id'],
      dtype='object')

In [70]:
new_columns_order = ['id', 'en', 'iso_code', 'Year', 'WRI',
                     'Exposure', 'Vulnerability', 'Susceptibility', 'Lack of Coping Capabilities',
                     ' Lack of Adaptive Capacities', 'Exposure Category',
                     'WRI Category', 'Vulnerability Category', 'Susceptibility Category']

english_dataframe = english_dataframe[new_columns_order]
english_dataframe

,id,en,iso_code,Year,WRI,Exposure,Vulnerability,Susceptibility,Lack of Coping Capabilities,Lack of Adaptive Capacities,Exposure Category,WRI Category,Vulnerability Category,Susceptibility Category
0,1,Vanuatu,NaN,2011,32.00,56.33,56.81,37.14,79.34,53.96,Very High,Very High,High,High
1,2,Bolivia (Plurinational State of),NaN,2011,5.16,9.34,55.23,43.45,80.64,41.61,Very Low,Low,High,High
2,3,Jordan,NaN,2011,5.13,11.50,44.61,24.18,67.33,42.33,Low,Low,Low,Medium
3,4,Iran (Islamic Republic of),NaN,2011,5.11,10.40,49.07,21.48,80.01,45.73,Low,Low,Medium,Low
4,5,Lebanon,NaN,2011,5.01,11.12,45.03,21.05,70.28,43.75,Low,Low,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1913,Seychelles,NaN,2021,4.89,11.94,40.97,18.23,64.82,39.86,Low,Low,Low,Low
1913,1914,New Zealand,NaN,2021,4.96,17.59,28.20,16.06,47.45,21.08,High,Low,Very Low,Very Low
1914,1915,Brazil,NaN,2021,4.97,11.35,43.80,22.68,76.22,32.51,Low,Low,Medium,Medium
1915,1916,Hungary,NaN,2021,5.07,15.24,33.25,16.07,58.89,24.78,High,Low,Very Low,Very Low


In [ ]:
rename = ['id','country_name','iso_code','year','wri','exposure','vulnerability','susceptibility','coping_inability',
     'adaptive_inability','wri_category','exposure_category','vulnerability_category','susceptibility_category']

database_insert_df  = english_dataframe.copy()
database_insert_df.set_axis(rename, axis=1,inplace=True)
database_insert_df.to_csv('static/data/english_dataset.csv',index=False)
database_insert_csv = pd.read_csv('static/data/english_dataset.csv')
database_insert_csv